In [ ]:
!pip install tensorflow keras-tuner

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from keras_tuner.tuners import Hyperband
import pandas as pd
import os

In [ ]:
# Membaca file atribut
attr_df = pd.read_csv('../input/celeba-dataset/list_attr_celeba.csv')

# Mengatur path dataset
base_dir = '../input/celeba-dataset/img_align_celeba'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

# ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    base_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    subset='validation'
)

In [ ]:
def build_model(hp):
    base_model = ResNet50(include_top=False, input_shape=(224, 224, 3))
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu')(x)
    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model


In [ ]:
tuner = Hyperband(build_model, objective='val_accuracy', max_epochs=10, directory='my_dir', project_name='resnet_tuning')
tuner.search(train_generator, epochs=10, validation_data=val_generator)
best_model = tuner.get_best_models(num_models=1)[0]


In [ ]:
history = best_model.fit(train_generator, epochs=50, validation_data=val_generator)

In [ ]:
val_loss, val_acc = best_model.evaluate(val_generator)
print(f"Validation Accuracy: {val_acc}")

In [ ]:
best_model.save('best_resnet_model.h5')